In [1]:
from sympy.ntheory.factor_ import factorint
from sympy.ntheory.generate import sieve, primerange, prevprime
from math import isqrt
from gmpy2 import iroot
from itertools import count
sieve.extend(10_000)

def primepi_naive(x):
    return len(list(sieve.primerange(x+1)))

primepi = primepi_naive

def phi(x, y):
    return 1 + len([n for n in range(2, x + 1) if y < min(factorint(n).keys())])

命題1 : $\pi(x) - \pi(\sqrt{x}) + 1 = \phi(x, \sqrt{x})$

In [2]:
for x in range(1, 300):
    sq_x = isqrt(x)
    assert primepi(x) - primepi(sq_x) + 1 == phi(x, sq_x)

In [3]:
def phi(x, y, k=None):
    if k is None:
        return 1 + len([n for n in range(2, x + 1) if y < min(factorint(n).keys())])
    if k == 0:
        return 1
    cnt = 0
    for n in range(2, x + 1):
        factors = factorint(n)
        if sum(factors.values()) == k and y < min(factors.keys()):
            cnt += 1
    return cnt

命題2 : $\phi(x, y) = \phi_0(x, y) + \phi_1(x, y) + \phi_2(x, y) + \cdots$

ここで、$y^k \ge x$ なら $\phi_k(x,y) = 0$

In [4]:
for x in range(1, 100):
    for y in primerange(x):
        p1 = phi(x, y)
        p2 = 0
        for k in count():
            if y**k >= x:
                break
            p2 += phi(x, y, k)
        assert p1 == p2, (x, y)

系3:
* $\phi(x, x^{1/2}) = 1 + \pi(x) - \pi(x^{1/2})$ (命題1)
* $\phi(x, x^{1/3}) = 1 + \pi(x) - \pi(x^{1/3}) + \phi_2(x, x^{1/3})$
* $\phi(x, x^{1/4}) = 1 + \pi(x) - \pi(x^{1/4}) + \phi_2(x, x^{1/4}) + \phi_3(x, x^{1/4})$

In [5]:
for x in range(1, 300):
    sq3_x = int(iroot(x, 3)[0])
    assert phi(x, sq3_x) == 1 + primepi(x) - primepi(sq3_x) + phi(x, sq3_x, 2)
    
for x in range(1, 300):
    sq4_x = int(iroot(x, 4)[0])
    assert phi(x, sq4_x) == 1 + primepi(x) - primepi(sq4_x) + phi(x, sq4_x, 2) + phi(x, sq4_x, 3)

命題4:

\begin{align*}
\phi_2(x, x^{1/3}) = {\pi(x^{1/3}) \choose 2} - {\pi(x^{1/2}) \choose 2} + \sum_{x^{1/3}< p \le x^{1/2}} \pi(x / p)
\end{align*}

In [6]:
def prop4_naive(x):
    """ \phi_2(x, x^{1/3}) を計算する
    """
    sq2_x = isqrt(x)
    sq3_x = int(iroot(x, 3)[0])
    c1 = primepi(sq3_x)
    c1 = c1*(c1 - 1) // 2
    c2 = primepi(sq2_x)
    c2 = c2*(c2 - 1) // 2
    c3 = 0
    for p in primerange(sq3_x+1, sq2_x + 1):
        c3 += primepi(x // p)
    return c1 - c2 + c3

In [7]:
for x in range(1, 300):
    assert prop4_naive(x) == phi(x, int(iroot(x, 3)[0]), 2)

$p_a$を$a$番目の素数とする。($p_1 = 2, p_2 = 3, p_3 = 5$)

命題5 : $\phi(x, p_a) = \phi(x, p_{a-1}) - \phi(x / p_a, p_{a-1})$


In [8]:
for x in range(1, 100):
    for p in primerange(3, x):
        assert phi(x, p) == phi(x, prevprime(p)) - phi(x // p, prevprime(p))

命題6 : 
\begin{align*}
\phi(x, p_a) = \sum_{n \mid p_2p_3\dots p_a} \mu(n)\left\lfloor \frac{x/n + 1}{2} \right\rfloor
\end{align*}

In [9]:
def prop6_naive(x, p):
    s = (x+1) >> 1
    primes = list(primerange(3, p+1))
    stack = [(0, 1, 1)]
    while stack:
        idx, n, mu = stack.pop()
        np = n*primes[idx]
        if x < np:
            continue
        idx += 1
        if idx < len(primes):
            stack.append((idx, n, mu))
            stack.append((idx, np, -mu))
        s -= mu * ((x // np + 1) // 2)
    return s

In [10]:
for x in range(1, 100):
    for p in primerange(3, x):
        assert phi(x, p) == prop6_naive(x, p)